In [1]:
import os
import math
import sys
import time
import copy
from tqdm import tqdm

import cv2

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import seaborn as sns


In [2]:
"""
- get responsible files for our target time durations (e.g., 7am to 9am)
"""
DATA_COLUMNS_NAME = ['taxiID', 'lng', 'lat', 'alt', 'time', 'azim', 'vel', 'gpstype', 'occ']

DATA_BASEDIR = "/home/gskim/data/CE545 Taxi/data"

TARGET_DATE = "2018-04-02"
TARGET_DURATION = ["070000", "100000"]

taget_date_dir = os.path.join(DATA_BASEDIR, TARGET_DATE)
taget_date_files = os.listdir(taget_date_dir)
taget_date_files.sort()

taget_date_files_in_duration = []
for filename in taget_date_files:
    if TARGET_DURATION[0] < filename[:-4] and filename[:-4] < TARGET_DURATION[1]:
        taget_date_files_in_duration.append(filename)
print(taget_date_files_in_duration)
print("\nThe number of files: ", len(taget_date_files_in_duration))

['070230.DAT', '070500.DAT', '070730.DAT', '071000.DAT', '071230.DAT', '071500.DAT', '071730.DAT', '072000.DAT', '072230.DAT', '072500.DAT', '072730.DAT', '073000.DAT', '073230.DAT', '073500.DAT', '073730.DAT', '074000.DAT', '074230.DAT', '074500.DAT', '074730.DAT', '075000.DAT', '075230.DAT', '075500.DAT', '075730.DAT', '080000.DAT', '080230.DAT', '080500.DAT', '080730.DAT', '081000.DAT', '081230.DAT', '081500.DAT', '081730.DAT', '082000.DAT', '082230.DAT', '082500.DAT', '082730.DAT', '083000.DAT', '083230.DAT', '083500.DAT', '083730.DAT', '084000.DAT', '084230.DAT', '084500.DAT', '084730.DAT', '085000.DAT', '085230.DAT', '085500.DAT', '085730.DAT', '090000.DAT', '090230.DAT', '090500.DAT', '090730.DAT', '091000.DAT', '091230.DAT', '091500.DAT', '091730.DAT', '092000.DAT', '092230.DAT', '092500.DAT', '092730.DAT', '093000.DAT', '093230.DAT', '093500.DAT', '093730.DAT', '094000.DAT', '094230.DAT', '094500.DAT', '094730.DAT', '095000.DAT', '095230.DAT', '095500.DAT', '095730.DAT']

The 

In [3]:
"""
- merge all (in that time zone) files 
- make data frame per each car 
"""

dfs = []
for filename in taget_date_files_in_duration:
    # read
    df_path = os.path.join(taget_date_dir, filename)
    df = pd.read_csv(df_path)
    df.columns = DATA_COLUMNS_NAME  

    # add
    dfs.append(df)

# merge and make all-in-one dataframe   
df_all = pd.concat(dfs)   

# for memory saving 
del dfs
df_all = df_all.drop("gpstype", axis=1)

In [4]:
df_all

,taxiID,lng,lat,alt,time,azim,vel,occ
0,180725800,1266741600,375593633,600,20180402070419,89,0,0
1,180725800,1266741600,375593616,599,20180402070429,89,0,0
2,180725800,1266741600,375593616,599,20180402070439,89,0,0
3,180725800,1266741583,375593616,598,20180402070449,89,0,0
4,180782803,1268622466,375493900,972,20180402070233,87,15,0
5,180782803,1268630016,375488666,614,20180402070243,187,20,0
6,180782803,1268630450,375481316,421,20180402070253,187,15,0
7,180782803,1268630766,375480000,309,20180402070303,208,5,0
8,180782803,1268629216,375471733,272,20180402070313,187,41,0
9,180782803,1268626049,375460416,261,20180402070323,202,20,0


In [6]:
# sort by taxi ID
# df_all = df_all.sort_values(["taxiID"], ascending=[True])
# df_all = df_all.reset_index(drop=True)

# grouping by taxi ID
taxi_grouped = df_all.groupby('taxiID')
taxi_grouped.count()

,lng,lat,alt,time,azim,vel,occ
taxiID,,,,,,,
180000525,1.265273e+09,3.748992e+08,0.000000,2.018040e+13,164.280952,61.961905,0.600000
180000826,1.271420e+09,3.753592e+08,0.000000,2.018040e+13,157.797933,18.025258,0.471871
180002092,1.270205e+09,3.751715e+08,0.000000,2.018040e+13,193.544928,19.682126,0.745894
180002578,1.272082e+09,3.739424e+08,0.000000,2.018040e+13,123.032258,37.338710,0.000000
180003216,1.271041e+09,3.748409e+08,0.000000,2.018040e+13,203.161905,14.849524,0.677143
180100817,1.269461e+09,3.753953e+08,0.000000,2.018040e+13,173.239614,19.979710,0.799034
180103394,1.248714e+09,3.690215e+08,0.000000,2.018040e+13,126.739819,18.143665,0.930995
180104894,1.271026e+09,3.751394e+08,0.000000,2.018040e+13,189.372947,18.592271,0.637681
180105156,5.000000e+00,5.000000e+00,0.000000,2.027123e+13,255.166667,16.833333,0.000000


In [7]:
for key in taxi_grouped.groups:
    print(key)

180000525
180000826
180002092
180002578
180003216
180100817
180103394
180104894
180105156
180105334
180105720
180106437
180106932
180106955
180106961
180107104
180107721
180108440
180108764
180109496
180109650
180109673
180110743
180110780
180111612
180111675
180111706
180111823
180111878
180111942
180112462
180113693
180113701
180113949
180114342
180114425
180114952
180115014
180116465
180117015
180117285
180117480
180117556
180118197
180118715
180120251
180120264
180120910
180122322
180122949
180123314
180124027
180124895
180125052
180125430
180125483
180125541
180126108
180126110
180126620
180126925
180127086
180127439
180127568
180128144
180201799
180202069
180202525
180203046
180203429
180203610
180204262
180204513
180204725
180204782
180204801
180205678
180206209
180207008
180207469
180209792
180210259
180211112
180213044
180213076
180213783
180213864
180213968
180214095
180214102
180215470
180215656
180216965
180217310
180218052
180218440
180218518
180219248
180219610
180220407


180711792
180711793
180711794
180711796
180711801
180711802
180711804
180711805
180711806
180711809
180711810
180711813
180711814
180711818
180711819
180711821
180711827
180711828
180711832
180711833
180711837
180711838
180711839
180711840
180711842
180711845
180711850
180711852
180711853
180711854
180711857
180711859
180711860
180711862
180711863
180711864
180711865
180711867
180711878
180711880
180711886
180711888
180711889
180711890
180711892
180711894
180711895
180711896
180711899
180711901
180711904
180711906
180711907
180711908
180711909
180711912
180711914
180711915
180711919
180711920
180711921
180711922
180711925
180711927
180711928
180711933
180711935
180711937
180711942
180711945
180711946
180711951
180711952
180711953
180711956
180711958
180711961
180711962
180711963
180711966
180711967
180711968
180711972
180711975
180711976
180711981
180711987
180711988
180711992
180711993
180711995
180711997
180711998
180711999
180712000
180712002
180712004
180712005
180712006
180712007


180722186
180722188
180722190
180722193
180722195
180722196
180722198
180722200
180722202
180722207
180722208
180722211
180722212
180722214
180722215
180722217
180722218
180722220
180722221
180722223
180722224
180722226
180722228
180722230
180722231
180722232
180722235
180722236
180722237
180722238
180722239
180722241
180722242
180722244
180722245
180722250
180722253
180722257
180722258
180722259
180722261
180722262
180722263
180722265
180722267
180722269
180722272
180722273
180722274
180722275
180722277
180722278
180722279
180722280
180722282
180722284
180722286
180722287
180722288
180722289
180722291
180722292
180722293
180722296
180722297
180722300
180722301
180722303
180722304
180722305
180722306
180722307
180722317
180722318
180722319
180722320
180722323
180722324
180722325
180722326
180722327
180722329
180722332
180722333
180722334
180722335
180722338
180722339
180722342
180722343
180722346
180722347
180722348
180722349
180722351
180722353
180722354
180722355
180722356
180722357


180737689
180737690
180737694
180737695
180737700
180737701
180737702
180737706
180737707
180737708
180737709
180737712
180737713
180737718
180737721
180737722
180737723
180737726
180737728
180737732
180737733
180737735
180737736
180737741
180737742
180737744
180737746
180737747
180737750
180737751
180737753
180737754
180737755
180737756
180737757
180737759
180737760
180737761
180737762
180737763
180737771
180737773
180737774
180737775
180737777
180737779
180737780
180737782
180737783
180737785
180737787
180737788
180737790
180737793
180737794
180737797
180737798
180737799
180737802
180737804
180737807
180737808
180737810
180737814
180737819
180737820
180737821
180737822
180737824
180737828
180737830
180737833
180737836
180737837
180737844
180737845
180737846
180737847
180737848
180737849
180737859
180737861
180737862
180737872
180737873
180737875
180737878
180737879
180737880
180737882
180737883
180737884
180737885
180737886
180737889
180737891
180737896
180737897
180737898
180737899


180754161
180754162
180754164
180754165
180754166
180754168
180754170
180754172
180754177
180754181
180754183
180754184
180754185
180754186
180754187
180754188
180754189
180754190
180754191
180754192
180754194
180754196
180754199
180754206
180754209
180754217
180754218
180754221
180754222
180754223
180754226
180754227
180754228
180754233
180754236
180754237
180754240
180754242
180754244
180754245
180754247
180754255
180754256
180754259
180754265
180754266
180754267
180754268
180754270
180754272
180754273
180754274
180754276
180754277
180754278
180754279
180754280
180754281
180754282
180754284
180754286
180754287
180754288
180754289
180754292
180754293
180754297
180754299
180754300
180754301
180754302
180754303
180754307
180754310
180754311
180754312
180754315
180754316
180754319
180754320
180754322
180754325
180754327
180754328
180754329
180754330
180754331
180754332
180754333
180754336
180754338
180754341
180754343
180754347
180754349
180754350
180754351
180754352
180754354
180754355


KeyboardInterrupt: 

In [ ]:
# fig, ax = plt.subplots(figsize=(10, 10))
# ax = sns.lineplot(x="time", y="occ", data=taxi)
# plt.xlabel('passenger on and off graph ', fontsize=15)
# plt.show()


In [ ]:
# print(np.count_nonzero(taxi_np_diff == EVENT_ON_TO_OFF), "passengers got off the taxi ID", taxi_id)  
#     print(np.count_nonzero(taxi_np_diff == EVENT_OFF_TO_ON), "passengers took the taxi ID", taxi_id)

EVENT_OFF_TO_ON = -1
EVENT_ON_TO_OFF = 1

taxi_id_dict = taxi_grouped.groups

for for_idx, taxi_id in enumerate(taxi_id_dict):
    taxi = taxi_grouped.get_group(taxi_id)
    taxi = taxi.sort_values(["time"], ascending=[True])
    taxi = taxi.reset_index(drop=True)

    taxi_np = taxi.values
    taxi_np = taxi_np[1:, :]
    taxi_np_occ_diff = taxi_np[1:, -1] - taxi_np[:-1, -1]
    for occ_idx, occ_flag in enumerate(taxi_np_occ_diff):
        if(occ_flag == EVENT_OFF_TO_ON):
#             print("A passenger took the taxi (" + str(taxi_id) + ")", 
#                   "at time", taxi_np[occ_idx, 4],
#                   "and (lng, lat) =", taxi_np[occ_idx, 1], taxi_np[occ_idx, 2])
#             occ_time_in = taxi_np[occ_idx, 4]

        if(occ_flag == EVENT_ON_TO_OFF):
#             print("A passenger got off the taxi (" + str(taxi_id) + ")", 
#                   "at time", taxi_np[occ_idx, 4],
#                   "and (lng, lat) =", taxi_np[occ_idx, 1], taxi_np[occ_idx, 2])
#             occ_time_out = taxi_np[occ_idx, 4]
            
#             minute_diff = float(str(occ_time_out)[-4:-2]) - float(str(occ_time_in)[-4:-2])
#             print(minute_diff, "minuites.\n")


In [ ]:
# print(np.count_nonzero(taxi_np_diff == EVENT_ON_TO_OFF), "passengers got off the taxi ID", taxi_id)  
#     print(np.count_nonzero(taxi_np_diff == EVENT_OFF_TO_ON), "passengers took the taxi ID", taxi_id)

EVENT_OFF_TO_ON = 1
EVENT_ON_TO_OFF = -1

taxi_id_dict = taxi_grouped.groups

taxi_on_history_start_flag = 0
taxi_off_history_start_flag = 0
for for_idx, taxi_id in enumerate(tqdm(taxi_id_dict, mininterval=20)):                

    if(for_idx % 10000 == 0):
        print(taxi_on_history.shape)
#         print(taxi_off_history.shape)

    taxi = taxi_grouped.get_group(taxi_id)
    taxi = taxi.sort_values(["time"], ascending=[True])
    taxi = taxi.reset_index(drop=True)

    taxi_np = taxi.values
    taxi_np = taxi_np[1:, :]
    taxi_np_occ_diff = taxi_np[:-1, -1] - taxi_np[1:, -1] # EVENT_OFF_TO_ON = 1, EVENT_ON_TO_OFF = -1
    for occ_idx, occ_flag in enumerate(taxi_np_occ_diff.tolist()):

        if(occ_flag == EVENT_OFF_TO_ON):
            event = taxi_np[occ_idx, :]
            event = np.expand_dims(event, axis=0)
            if(taxi_on_history_start_flag == 0):
                taxi_on_history = event
                taxi_on_history_start_flag = 1
            else:
                taxi_on_history = np.concatenate((taxi_on_history, event), axis=0)
                                
        if(occ_flag == EVENT_ON_TO_OFF):
            event = taxi_np[occ_idx, :]
            event = np.expand_dims(event, axis=0)
            if(taxi_off_history_start_flag == 0):
                taxi_off_history = event
                taxi_off_history_start_flag = 1
            else:
                taxi_off_history = np.concatenate((taxi_off_history, event), axis=0)
                


In [ ]:
taxi_on_history_ = taxi_on_history[:, :-1]
taxi_off_history_ = taxi_off_history[:, :-1]

print(taxi_on_history.shape)
print(taxi_off_history.shape)

np.savetxt("taxi_on_20180402_070000_100000.csv", taxi_on_history_, delimiter=",")
np.savetxt("taxi_off_20180402_070000_100000.csv", taxi_off_history_, delimiter=",")
